In [11]:
import numpy as np
import pandas as pd
import os
import cv2
import time
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import skimage
import keras
from keras.layers import Conv2D,Activation,MaxPooling2D,AveragePooling2D,BatchNormalization,Dropout,Flatten,Dense
from keras.models import Model,Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16,preprocess_input
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.layers import LeakyReLU
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
X=[]
Z=[]
IMG_SIZE=150
AB_DIR = 'C:\\Users\\Tanishqa Mahendru\\Desktop\\proj\\woundData(Updated)\\abrasion(Updated)'
LAC_DIR = 'C:\\Users\\Tanishqa Mahendru\\Desktop\\proj\\woundData(Updated)\\lacerated(Updated)'
BU_DIR = 'C:\\Users\\Tanishqa Mahendru\\Desktop\\proj\\woundData(Updated)\\Burn(Updated)'
from tqdm import tqdm
import os      
import cv2
import numpy as np

def assign_label(img,wtype):
    return wtype

def make_train_data(wtype,DIR):
    for img in tqdm(os.listdir(DIR)):
        label=assign_label(img,wtype)
        path = os.path.join(DIR,img)
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        
        X.append(np.array(img))
        Z.append(str(label))
        
make_train_data('Abrasion',AB_DIR)
make_train_data('Laceration',LAC_DIR)
make_train_data('Burn',BU_DIR)
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()
Y=le.fit_transform(Z)
Y=to_categorical(Y,3)
X=np.array(X)
X=X/255
X.shape

100%|███████████████████████████████████████| 172/172 [00:00<00:00, 326.98it/s]


(378, 150, 150, 3)

In [12]:
base_model = ResNet50(weights='imagenet',include_top = False,input_shape=(150,150,3))
base_model.summary()

c:\users\tanishqa mahendru\appdata\local\programs\python\python37\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 150, 150, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 156, 156, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 75, 75, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 75, 75, 64)   256         conv1[0][0]                      
___________________________________________________________________________________________

In [13]:

num_classes = 3
batchsize = 20
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(1024))
model.add(LeakyReLU(0.01))
model.add(Dropout(0.5))
model.add(Dense(num_classes,activation='softmax'))
model.summary()
optimizer = keras.optimizers.adam(lr = 0.001)
#model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['acc'])
model.compile(optimizer=SGD(lr=0.002, momentum=0.0, decay=0.0, nesterov=True), loss='mse', metrics=['accuracy'])
#model.compile(loss='binary_crossentropy',optimizer=optimizers.RMSprop(lr=1e-4), metrics=['accuracy'])

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=4)
data_generator = ImageDataGenerator(width_shift_range=0.1,height_shift_range=0.1,rotation_range=15,zoom_range=0.1 )
#data_generator = ImageDataGenerator( rotation_range=25,width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, horizontal_flip=True)
model.fit_generator(data_generator.flow(X_train,y_train,batch_size = batchsize), steps_per_epoch=int(len(X_train)/batchsize), epochs=10)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 5, 5, 2048)        23587712  
_________________________________________________________________
flatten_4 (Flatten)          (None, 51200)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 1024)              52429824  
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 1024)              0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 3075      
Total params: 76,020,611
Trainable params: 75,967,491
Non-trainable params: 53,120
_____________________________________

In [14]:
score = model.evaluate(X_test, y_test, verbose=0)
print(score)

[0.08980246594077662, 0.8026315758102819]


In [23]:
#from tensorflow import liteimport numpy as np
from tensorflow import keras

import numpy as np
from tensorflow import keras
from tensorflow import lite
model.save('tl1.h5')


In [24]:
from tensorflow import lite

converter = lite.TocoConverter.from_keras_model_file('tl1.h5')
tflite_model = converter.convert()
open("tl1.tflite", "wb").write(tflite_model)

W1108 00:44:22.630806  7152 hdf5_format.py:258] Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.


303681264